In [5]:
from unet_xception_model import get_model

IMG_SIZE = (450, 600)

my_model = get_model(img_size=(160, 160), in_channels=3, classes=3)
my_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_9 (Conv2D)              (None, 80, 80, 32)   896         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_15 (BatchN  (None, 80, 80, 32)  128         ['conv2d_9[0][0]']               
 ormalization)                                                                                    
                                                                                            